In [1]:
import numpy as np
import pandas as pd
import os

from astropy.io import fits
from collections import OrderedDict

from bokeh.models import HoverTool, TapTool, ColorBar
from bokeh.models import Circle, Range1d, ColumnDataSource, Span, CustomJS
from bokeh.models.glyphs import Image, ImageURL
from bokeh.models.mappers import LinearColorMapper
from bokeh.embed import components, file_html
from bokeh.palettes import RdBu11
from bokeh.plotting import gridplot, figure
from bokeh.plotting import show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
def colormap(col, palette):
    n,bins = np.histogram(col[np.isfinite(col)],bins=len(palette) - 1)
    color = col.copy().astype(str)
    binsize = bins[1] - bins[0]
    for num, val in enumerate(bins):
        cond = (col > val - binsize) & (col <= val)
        color[cond] = palette[num]
    color[color=='nan'] = '#999999'
    return color

In [3]:
band_labels = ['j', 'h', 'k', '3', '4']
band_names = ['J', 'H', 'K_s', '[3.6]', '[4.5]']
# cmap = plt.cm.get_cmap('gist_rainbow')
# plot_colors = list(reversed([cmap(i) for i in np.linspace(0,0.91,len(band_labels))]))
mag_offset = [8, 6, 4, 2, 0]
wavelength = [1.220, 1.630, 2.190, 3.550, 4.493]

In [4]:
df = pd.read_csv('../reworked_fitting_code/final_data_files/all_possible_photometry.csv')
df['logP'] = np.log10(df.per)
df['type_vowel'] = df.type.astype(str).str.replace('0','RRab').replace('1','RRc')
color = colormap(df.photfeh,RdBu11)
df['color'] = color

In [12]:
logP = np.log10(df.per)
source = ColumnDataSource(data=df.astype(str).to_dict(orient='list'))
figure_dict = OrderedDict()
for i in range(5):
    p = figure(y_range=(14.6, 11.9), x_range=(-0.64, 0.06), plot_width=550)
    l = band_labels[i]
    source.add(list(df['mag_{}'.format(l)]-mag_offset[i]), name='mag__{}'.format(l))
    p.circle('logP', 'mag_{}'.format(l),color='color',
         source=source, size=7, line_color='black',
             line_width=0.7, fill_alpha=0.7, line_alpha=0.7)
    hover = HoverTool(tooltips=OrderedDict([('ID','@id'),('Type','@type_vowel'),('Per','@per'),
                                            ('[Fe/H]','@photfeh'),
                                            ('RA','@ra'),('Dec','@dec')]))
    p.add_tools(hover)
    p.yaxis.axis_label = '{} mag'.format(band_names[i])
    if i == 0:
        p.xaxis.major_label_text_font_size = '0pt'
        p.plot_height = 180
    elif i == 4:
        p.plot_height = 190
    else:
        p.xaxis.major_label_text_font_size = '0pt'
        p.plot_height = 150
    figure_dict['p{}'.format(i)] = p
    if i > 0:
        figure_dict['p{}'.format(i)].x_range = figure_dict['p0'].x_range
        figure_dict['p{}'.format(i)].y_range = figure_dict['p0'].y_range
figure_dict['p0'].title.text = 'Omega Cen RRL PL Relations'
figure_dict['p4'].xaxis.axis_label = 'log P (days)'
# color_bar = ColorBar(color_mapper=LinearColorMapper(palette="RdBu11",low=df.photfeh.min(),high=df.photfeh.max()),
#                      border_line_color=None, height=750)
# colorfig = figure(plot_height=750,plot_width=50)
# colorfig.add_layout(color_bar)
grid = gridplot(figure_dict.values(),ncols=1)
#grid.add_layout(color_bar, 'right')
#print p.select('pl')
# p.title.text_font_size='14pt'
# p.xaxis.axis_label_text_font_size = '12pt'
# p.yaxis.axis_label_text_font_size = '12pt'


#     p.circle('logP_{}'.format(l), 'mag__{}'.format(l),
#              source=source, size=7, line_color='black', line_width=0.4, fill_alpha=0.7, line_alpha=0.7,
#              name='pl_{}'.format(l))
show(grid)

In [ ]:
def plot(df, palette=RdBu11):
    logP = np.log10(df['per'])
    # add color to thing
    color = colormap(df['photfeh'],palette)
    # make ColumnDataSource
    source = ColumnDataSource(data=df.to_dict())
    source.add(logP, name='logP')
    source.add(color, name='color')
    # set yrange values
    
    p = figure(y_range=(ymax, ymin), plot_width=600, plot_height=750)
    hover = HoverTool(tooltips=OrderedDict([('ID','@id'),('Type','@type'),('Per','@per'),
                                            ('[Fe/H]','@photfeh'),('Zphot','@zphot')
                                            ('RA','@RAdeg{1.11111}'),('Dec','@DECdeg{1.11111}')]))
    p.add_tools(hover)
    
    p.circle('logP', 'mag_j', source=source, size=7, fill_color='color',
             line_color='black', line_width=0.4, fill_alpha=0.7, line_alpha=0.7, name='pls')
    
    #custom glyphs for selected + unselected points
    selected = Circle(size=7, fill_color='color', line_color='black',
                      line_width=0.4, fill_alpha=1.0, line_alpha=1.0)
    unselected = Circle(size=7, fill_color='color', line_color='black',
                         line_width=0.4, fill_alpha=0.1, line_alpha=0.5)
    renderer = p.select(name="pls")
    renderer.selection_glyph = selected
    renderer.nonselection_glyph = unselected

    # other plot junk
    p.title = 'Omega Cen PL Relations'
    p.xaxis.axis_label = 'log P (days)'
    p.yaxis.axis_label = 'Apparent Magnitude'
    p.title_text_font_size='14pt'
    p.xaxis.axis_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    # return plot object
    return p


In [ ]:
{'image{}'.format(i+1):band_names[i] for i in range(5)}